# BOD 1 - Návrh zadania

## 1. Motivácia
Počasie patrí medzi najvýraznejšie faktory, ktoré vplývajú na ľudský život na Zemi. Človek sa odjakživa snažil čo najpresnejšie predpovedať počasie práve kvôli jeho veľkému vplyvu na život každého jedinca. Od predpovedi počasia sú v súčasnosti takisto závislé mnohé odvetvia, či už poľnohospodárstvo, doprava(letecká, lodná), výroba elektrickej energie (veterné elektrárne a solárne panely) a mnohé ďalšie. Z týchto dôvodov je čo najpresnejšia predpoveď počasia veľmi dôležitá.


V posledných rokoch sa začala na predpoveď počasia využívať nová technika - predpoveď pomocou neurónových sietí. Pri predpovedi počasia ide prakticky o predikciu časových radov. Kvôli premenlivosti sa počasia sa často na tieto problémy využívajú rekurentné neurónové siete.


V našom projekte sme sa rohodli zaoberať predikciou časových radov a teda predpoveďou počasia. Našim cieľom je využiť rekurentné neurónové siete. Budeme pracovať s dátami zo štyroch Amerických miest ležiacich na Juhozápade Spojených štátov:

- Los Angeles
- San Francisco
- Phoenix
- Las Vegas

## 2. Súvisiaca práca
- *Sequence to Sequence Weather Forecasting with Long Short-Term Memory Recurrent Neural Networks 2016*


V tejto práci sa autori zaoberajú predikciou počasia pre 9 miest v Maroku. Snažia sa o predpoveď teploty, vlhkosti a rýchlosti vetra na dva časové intervaly - 24 a 72 hodín. Používajú pri tom LSTM neurónové siete, ktoré patria do skupiny rekurentných neurónových sietí.

Na natrénovanie modelu neorónovej siete použili približne 15 rokov meteorologických dát, ktoré boli zaznamenané v časovom intervale jednej hodiny. Model dosiahol dobrú úspešnnosť predpovedí, porovnateľnú s úspešnosťou metód, ktoré sa na predpoveď počasia používajú v dnešnej dobe. Hodnoty sú v minutových intervaloch a zaokrúhlené na tri desatinné miesta.

Ďalšie práce v tejto oblasti:
- *An ensemble of neural networks for weather forecasting 2004*
- *An Efficient Weather Forecasting System using Artificial Neural Network 2010*
- *Weather Forecasting using Deep Learning Techniques 2015*
- *An effective weather forecasting using neural network 2014*


## 3. Dataset
Dataset obsahuje teplotu zaznamenanú vo vyššie uvedených mestách mestách v hodinových intervaloch počas obdobia jedého roka. Dokopy tak máme 35036 záznamov, 8759 pre každé mesto.

Ako bolo uvedené v analýze dát, dáta sú kompletné, neobsahujú chýbajúce hodnoty ani vychýlené hodnoty(outliery).

### Príklad dát:
![weather](./images/pocasiedata.png)


## 4. Návrh riešenia
Náš model sa bude skladať z viacvrstvovej rekurentnej neurónovej siete. Túto sieť budeme trénovať na vopred opísanom datasete a chceme aby predikovala teplotu a iné meteorologické dáta na pár dní dopredu.

# BOD 2 - Analýza dát

Náš vstupný dataset sa skladá z dát z troch Amerických miest:

- Los Angeles
- Phoenix
- San Francisco

Všetky tieto mestá ležia blízko pri sebe na juhozápadnom pobreží Spojených štátov čo znamená, že sa počasie v týchto mestách veľmi nelíši. Počasie je veľmi ovplyvňované Tichým oceánom.

Náš dataset obsahuje údaje o priemernej teplote(uvádzané vo Fahrenheitoch) v hodinovách intervaloch. Tieto údaje boli zozbierané počas jedného roka . Pre každé mesto máme 8759 záznamov. 

V projekte budeme pracovať na predikcií teploty.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sea
from scipy.stats import pearsonr, spearmanr
from pandas.plotting import scatter_matrix
%matplotlib inline

## Práca s dátami

Vstupné dáta z datasetu importujeme do Pandas dataframu, ktorý využijeme na ďalšiu analýzu. Nižšie môžeme vidieť základné údaje o datasete.

In [ ]:
df = pd.read_csv('temperatures.csv')
df.shape

In [ ]:
print(df.head)

In [ ]:
df.info()

## Základné štatistiky

V poliach nižšie môžeme vidieť základné štatistiky o dátach ako minimálne a maximálne hodnoty, priemer, medián a iné.

Na údaji o počte riadkov môžeme vidieť, že všetky dátav datasete sú vyplnené a dopĺňanie chýbajúcich hodnôt nebude potrebné.

V štatistikách o minimálnych a maximálnych teplotách môžeme vidieť niekoľko zaujímavých faktov. Minimálne teploty vo všetkých troh mestách sú veľmi podobné, medzi 45 a 50 Fahrenheitami. Na druhej strane, maximálne teploty sa líšia o čosi viac. Maximálna teplota vo Phoenixe je o viac ako 30 Fahrenheitov vyššia ako maximálna teplota vo zvyšných dvoch mestách. Phoenix je mesto s najnižšou minimálnou a najvyššou maximálnou teplotou, čo zároveň znamená aj najväčší rozdiel medzi minimálnou a maximálnou teplotou.

In [ ]:
df.describe()

In [ ]:
la = df['LA-HLY-TEMP-NORMAL']
ph = df['PH-HLY-TEMP-NORMAL']
sf = df['SF-HLY-TEMP-NORMAL']

In [ ]:
np.median(la)

In [ ]:
np.median(ph)

In [ ]:
np.median(sf)

## Histogramy

V histogramoch nižšie môžeme vidieť najčastejšie a najmenej časté teploty počas roka.

Môžeme vidieť, že najčastejšie teploty v Los Angeles sa nachádzajú v rozmedzí okolo 65 Fahranheitov. Všetky teploty sú približne medzi 50 až 75 Fahrenheitami.

Teploty vo Phoenixe sú najvyššie spomedzi všetkých troch miest. Výskyt jednotlivých teplôt je rozložený rovnomernejšie. Najčastejší výskyt dosahujú teploty medzi 65 až 85 Fahrenheitami.

Distribúcia teplôt v San Franciscu je sprava zošikmená. Hodnoty teplôt sú medzi 45 až 75 Fahrenheitemi a najvyšší bod je okolo 55 Fahrenheitov.

#### Los Angeles

In [ ]:
sea.distplot(la, bins=50)

#### Phoenix

In [ ]:
sea.distplot(ph, bins=50)

#### San Francisco

In [ ]:
sea.distplot(sf, bins=50)

## Korelácie medzi premennými

V grafoch a štatistikách nižšie môžeme vidieť korelácie medzi jednotlivými premennými.

Všetky tri korelácie sú veľmi silne pozitívne s minimálnou hodnotou 0.85. Najviac pozitívna korelácia je medzi Los Angeles a San Franciscom s magnitúdou takmer 0.95.

#### Korelácia medzi Los Angeles a Phoenixom

In [ ]:
np.corrcoef(la, ph)

In [ ]:
plt.scatter(la, ph)
plt.show()

#### Korelácia medzi Los Angeles a San Franciscom

In [ ]:
np.corrcoef(la, sf)

In [ ]:
plt.scatter(la, sf)
plt.show()

#### Korelácia medzi San Franciscom a Phoenixom

In [ ]:
np.corrcoef(sf, ph)

In [ ]:
plt.scatter(sf, ph)
plt.show()

#### Porovnanie korelácií všetkých troch miest zároveň

In [ ]:
df.corr()

In [ ]:
scatter_matrix(df, figsize=(10, 10))
plt.show()

Matica korelácií všetkých troch miest:

0 - Los Angeles

1 - Phoenix

2 - San Francisco

In [ ]:
plt.matshow(df.corr())
plt.colorbar()
plt.show()

## Vychýlené hodnoty

Často používaná technika na detekciu vychýlených hodnôt je boxplot diagram. V diagramoch nižšie môžeme vidieť, že v ani jednom z troch miest nie sú vychýlené hodnoty. Rovnaké hodnoty môžeme vidieť na histogramoch vyššie.

#### Los Angeles

In [ ]:
sea.boxplot(x=la)

#### Phoenix

In [ ]:
sea.boxplot(x=ph)

#### San Francisco

In [ ]:
sea.boxplot(x=sf)

# BOD 3 - Záverečná časť

Do záverečnej časti sme sa rozhodli do datasetu pridať ešte údaje zo štvrtého mesta - Las Vegas. Toto mesto svojou geografickou polohou veľmi dobre zapadá medzi ostatné mestá.

In [ ]:
las_vegas = pd.read_csv("vegas_hourly.csv")
las_vegas.rename(columns={'HLY-TEMP-NORMAL': 'LV-HLY-TEMP-NORMAL'}, inplace=True)
los_angeles = pd.read_csv("losangeles_hourly.csv")
los_angeles.rename(columns={'HLY-TEMP-NORMAL': 'LA-HLY-TEMP-NORMAL'}, inplace=True)
phoenix = pd.read_csv("phoenix_hourly.csv")
phoenix.rename(columns={'HLY-TEMP-NORMAL': 'PH-HLY-TEMP-NORMAL'}, inplace=True)
san_francisco = pd.read_csv("sanfrancisco_hourly.csv")
san_francisco.rename(columns={'HLY-TEMP-NORMAL': 'SF-HLY-TEMP-NORMAL'}, inplace=True)
temperatures = pd.concat([las_vegas[['DATE','LV-HLY-TEMP-NORMAL']],los_angeles['LA-HLY-TEMP-NORMAL'],phoenix['PH-HLY-TEMP-NORMAL'],san_francisco['SF-HLY-TEMP-NORMAL']],axis=1,sort=False)
temperatures.to_csv('temperatures4.csv',index=False)

#### Mestá
Na mape sú farebne vyznačené mestá, z ktorých pochádzajú vstupné dáta.
![cities](./images/cities.png)

Počasie v mestách je pod vplyvom Tichého oceánu čo potvrdzuje nasledovná mapa veterných prúdov v oblasti.
![wind](./images/wind.jpg)

In [ ]:
def create_graph(filename):
    graph = pd.read_csv(filename,sep=';')
    plt.plot(graph.loss[1:], label='loss')
    plt.plot(graph.val_loss[1:],c ='r', label='val_loss')
    plt.legend(loc='best')
    plt.show()
    
create_graph("log_30_11_12-30-04.csv")

## Experimenty

Pri každom experimente uvádzame architektúru medelu neurónovej siete, postup pri trénovaní a dosiahnuté výsledky.

#### Experiment 1

Architektúra:

Hyperparametre pri trénovaní:

Výsedky:

Loss funkcie:

In [ ]:
create_graph("test1.csv.csv")

#### Experiment 

Architektúra:

Hyperparametre pri trénovaní:

Výsedky:

Loss funkcie